In [1]:
!git clone https://github.com/muxspace/facial_expressions.git

Cloning into 'facial_expressions'...
remote: Enumerating objects: 14214, done.
remote: Total 14214 (delta 0), reused 0 (delta 0), pack-reused 14214
Receiving objects: 100% (14214/14214), 239.65 MiB | 37.52 MiB/s, done.
Resolving deltas: 100% (223/223), done.
Updating files: 100% (13996/13996), done.


In [2]:
import csv
data={}
with open('/content/facial_expressions/data/legend.csv')as f:
  reader=csv.reader(f)
  next(reader)
  for row in reader:
    key=row[2]
    if key in data:
      data[key].append(row[1])
    else:
      data[key]=[row[1]]


In [3]:
emotion_list=list(data.keys())
emotion_list

['anger',
 'surprise',
 'disgust',
 'fear',
 'neutral',
 'happiness',
 'sadness',
 'contempt',
 'NEUTRAL',
 'SADNESS',
 'DISGUST',
 'FEAR',
 'SURPRISE',
 'ANGER',
 'HAPPINESS']

In [4]:
import os
os.mkdir('master_data')
os.mkdir('master_data/training')
os.mkdir('master_data/testing')

In [5]:
for emotion in emotion_list:
  os.mkdir(os.path.join('master_data/training/',emotion))
  os.mkdir(os.path.join('master_data/testing/',emotion))

In [6]:
from shutil import copyfile
split_size=0.8
for emotion,images in data.items():
  train_size=int(split_size*len(images))
  train_images=images[:train_size]
  test_images=images[train_size:]
  for image in train_images:
    source = os.path.join('/content/facial_expressions/images',image)
    dest = os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)
  for image in test_images:
    ource=os.path.join('/content/facial_expression/images',image)
    dest=os.path.join('/content/master_data/training',emotion,image)
    copyfile(source,dest)

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Conv2D,MaxPooling2D,Dense,Flatten

In [8]:
model=tf.keras.models.Sequential([
     Conv2D(16,(3,3),activation='relu',input_shape=(100,100,3)),
     MaxPooling2D(2,2),
     Conv2D(32,(3,3),activation='relu'),
     MaxPooling2D(2,2),
     Conv2D(64,(3,3),activation='relu'),
     MaxPooling2D(2,2),
     Flatten(),
     Dense(512,activation='relu'),
     Dense(15,activation='softmax')])
model.compile(optimizer=Adam(lr=0.01),loss='categorical_crossentropy',metrics=['acc'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 98, 98, 16)        448       
                                                                 
 max_pooling2d (MaxPooling2  (None, 49, 49, 16)        0         
 D)                                                              
                                                                 
 conv2d_1 (Conv2D)           (None, 47, 47, 32)        4640      
                                                                 
 max_pooling2d_1 (MaxPoolin  (None, 23, 23, 32)        0         
 g2D)                                                            
                                                                 
 conv2d_2 (Conv2D)           (None, 21, 21, 64)        18496     
                                                                 
 max_pooling2d_2 (MaxPoolin  (None, 10, 10, 64)        0

In [9]:
train_dir='/content/master_data/training'
test_dir='/content/master_data/testing'
train_datagen=ImageDataGenerator(rescale=1.0/255)
train_generator=train_datagen.flow_from_directory(train_dir,target_size=(100,100),class_mode='categorical',batch_size=128)

test_datagen=ImageDataGenerator(rescale=1.0/255)
test_generator=test_datagen.flow_from_directory(train_dir,target_size=(100,100),class_mode='categorical',batch_size=128)

Found 13683 images belonging to 15 classes.
Found 13683 images belonging to 15 classes.


In [10]:
es=EarlyStopping(monitor='val_acc',patience=2,min_delta=0.01)

In [11]:
model.fit(train_generator,
                    epochs=10,
                    verbose=1,
                    validation_data=test_generator,
                    callbacks=[es])

Epoch 1/10
107/107 [==============================] - 37s 295ms/step - loss: 0.9166 - acc: 0.6595 - val_loss: 0.5463 - val_acc: 0.8171
Epoch 2/10
107/107 [==============================] - 31s 291ms/step - loss: 0.5165 - acc: 0.8251 - val_loss: 0.5119 - val_acc: 0.8430
Epoch 3/10
107/107 [==============================] - 30s 285ms/step - loss: 0.4400 - acc: 0.8495 - val_loss: 0.3664 - val_acc: 0.8714
Epoch 4/10
107/107 [==============================] - 31s 295ms/step - loss: 0.3850 - acc: 0.8646 - val_loss: 0.3313 - val_acc: 0.8871
Epoch 5/10
107/107 [==============================] - 31s 289ms/step - loss: 0.3357 - acc: 0.8809 - val_loss: 0.2953 - val_acc: 0.8972
Epoch 6/10
107/107 [==============================] - 31s 293ms/step - loss: 0.3061 - acc: 0.8902 - val_loss: 0.2587 - val_acc: 0.9064
Epoch 7/10
107/107 [==============================] - 31s 289ms/step - loss: 0.2757 - acc: 0.8986 - val_loss: 0.2121 - val_acc: 0.9274
Epoch 8/10
107/107 [==============================] - 3